In [2]:
import sys
sys.path.append(r'C:\Calc\OpenIO_Data')
import openIO
import pandas as pd
import numpy as np

To get the data from openIO-Canada, you simply need to enter 3 arguments:
- the path to the STATCAN economic data (Detailed level) for the year you chose (available here: https://www150.statcan.gc.ca/n1/en/catalogue/15-602-X). **Make sure to download the Excel versions for the code to work properly.**
- the path to an IOT pxp folder of Exiobase. Exiobase data can be found here: https://doi.org/10.5281/zenodo.5589597
- a boolean whether you wish to endogenize capitals or not. The default option is that they are endogenized. For more information on endogenization of capitals, read these articles: (https://doi.org/10.1021/acs.est.8b02791)
(https://doi.org/10.1111/jiec.12931). **Note that endogenization introduces a lot of uncertainties, but that not endogenizing capitals will result in emission factors not including the impact of infrastructure and in an underestimation of total emissions at provincial or national level.**

In [3]:
tables = openIO.IOTables(folder_path=r'C:\Calc\OpenIO_Data\StatCan\2021\Detail level',
                         exiobase_folder='C:\Calc\OpenIO_Data\Exiobase\monetary\exiobase_3.8.2\IOT_2021_pxp',
                         endogenizing_capitals=False)

2025-09-13 17:07:31,615 - openIO-Canada - INFO - Reading all the Excel files...
2025-09-13 17:07:44,420 - openIO-Canada - INFO - Formatting the Supply and Use tables...
2025-09-13 17:08:29,750 - openIO-Canada - INFO - Modifying names of duplicated sectors...
2025-09-13 17:08:29,964 - openIO-Canada - INFO - Organizing final demand sectors...
2025-09-13 17:08:30,713 - openIO-Canada - INFO - Removing IOIC codes from index...
2025-09-13 17:08:30,792 - openIO-Canada - INFO - Balancing inter-provincial trade...
2025-09-13 17:09:31,241 - openIO-Canada - INFO - Pre-treatment of international trade data...
2025-09-13 17:09:48,253 - openIO-Canada - INFO - Linking international trade data to openIO-Canada...
2025-09-13 17:10:46,587 - openIO-Canada - INFO - Building the symmetric tables...
2025-09-13 17:10:54,913 - openIO-Canada - INFO - Linking openIO-Canada to Exiobase...
2025-09-13 17:17:45,247 - openIO-Canada - INFO - Extracting and formatting environmental data from the NPRI file...
2025-09-1

Run the calc() function which will calculate the life cycle impacts of each sector.

In [4]:
tables.calc()

OpenIO-Canada takes a while to run (~45 minutes) on a good machine. To avoid having to re-run the code everytime you want to use openIO-Canada, you can save the whole object in a pickle file.

In [5]:
import gzip
import pickle

with gzip.open('C://Users/11max/PycharmProjects/OpenIOCanada/v2.10/openIO_2021_not_endogenized.gz.pickle', 'wb') as file:
    pickle.dump(tables, file)